Base에서 실행

In [1]:
import torch

C:\Users\user\anaconda3\envs\cuda\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\anaconda3\envs\cuda\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\Users\user\anaconda3\envs\cuda\lib\site-packages\numpy\.libs\libopenblas.xwydx2ikjw2nmtwsfyngfuwkqu3lytcz.gfortran-win_amd64.dll
  stacklevel=1)


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

df_newsgroup = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/20news_dataset_clear/20newsgroup_preprocessed.csv', sep=';', usecols=['target', 'text_cleaned'])
df_newsgroup.rename(columns={'text_cleaned' : 'text'}, inplace=True)

le = LabelEncoder()
le.fit(df_newsgroup['target'].unique())

df_newsgroup['target'] = le.transform(df_newsgroup['target'])

X = df_newsgroup['text'].astype(str)
# y = tf.keras.utils.to_categorical(df_newsgroup['target'], num_classes=df_newsgroup['target'].nunique())
y=list(df_newsgroup['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_newsgroup['target'])

x_train=list(X_train)
x_test=list(X_test)

sos_x_train=[]
sos_x_test=[]
for sen in x_train:
    sos_x_train.append('<sos>'+sen)
for sen in x_test:
    sos_x_test.append('<sos>'+sen)

all_txt=sos_x_train+sos_x_test

from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_txt)

vocab_size =len(tokenizer.word_index)

x_train_encoded = tokenizer.texts_to_sequences(sos_x_train)
x_test_encoded = tokenizer.texts_to_sequences(sos_x_test)

max_len = 300

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_encoded, maxlen=max_len)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test_encoded, maxlen=max_len)

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs


class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads, dff, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(embedding_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(dff, activation="relu"),
             tf.keras.layers.Dense(embedding_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs) # 첫번째 서브층 : 멀티 헤드 어텐션
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output) # Add & Norm
        ffn_output = self.ffn(out1) # 두번째 서브층 : 포지션 와이즈 피드 포워드 신경망
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output) # Add & Norm


class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, vocab_size, embedding_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.pos_emb = tf.keras.layers.Embedding(max_len, embedding_dim)

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


embedding_dim = 256  # 각 단어의 임베딩 벡터의 차원
num_heads = 1  # 어텐션 헤드의 수
dff = 32  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기
num_transformer_blocks = 1

inputs = tf.keras.layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
x = embedding_layer(inputs)

for _ in range(num_transformer_blocks):
    transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
    x = transformer_block(x)

# transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
# x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(20, activation="softmax")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.summary()

y_train = np.array(y_train)
y_test = np.array(y_test)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300)]             0         
                                                                 
 token_and_position_embeddin  (None, 300, 256)         38745344  
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_block (Transfor  (None, 300, 256)         280864    
 merBlock)                                                       
                                                                 
 global_average_pooling1d (G  (None, 256)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_2 (Dropout)         (None, 256)               0     

In [2]:
x_val=x_train[:1000]
x_train=x_train[1000:]
y_val=y_train[:1000]
y_train=y_train[1000:]

In [3]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(x_train, y_train, batch_size=256, epochs=5, validation_data=(x_val, y_val))

print("테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))

Epoch 1/5
55/55 [==============================] - 4s 50ms/step - loss: 2.9845 - accuracy: 0.0943 - val_loss: 2.5314 - val_accuracy: 0.3820
Epoch 2/5
55/55 [==============================] - 3s 50ms/step - loss: 1.3373 - accuracy: 0.5964 - val_loss: 0.6321 - val_accuracy: 0.8250
Epoch 3/5
55/55 [==============================] - 3s 47ms/step - loss: 0.3212 - accuracy: 0.9105 - val_loss: 0.5303 - val_accuracy: 0.8550
Epoch 4/5
55/55 [==============================] - 2s 44ms/step - loss: 0.1067 - accuracy: 0.9719 - val_loss: 0.5119 - val_accuracy: 0.8650
Epoch 5/5
118/118 [==============================] - 0s 4ms/step - loss: 0.6496 - accuracy: 0.8656
테스트 정확도: 0.8656


In [4]:
np.average([86.8,86.7,87.3])

86.93333333333334